In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.preprocessing import LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/technocolabs_prosper/prosperLoanData.csv'

Mounted at /content/drive


In [3]:
df = pd.read_csv(path)

In [4]:
df['ListingKey'].nunique()

113066

In [5]:
df['ListingNumber'].nunique()

113066

In [6]:
df['LoanKey'].nunique()

113066

In [7]:
df['LoanNumber'].nunique()

113066

In [8]:
df['MemberKey'].nunique()

90831

#dropping duplicates

In [9]:
df.drop_duplicates(subset = 'ListingKey', inplace = True)
df.drop_duplicates(subset = 'ListingNumber', inplace = True)
df.drop_duplicates(subset = 'LoanKey', inplace = True)
df.drop_duplicates(subset = 'LoanNumber', inplace = True)
df.drop_duplicates(subset = 'MemberKey', inplace = True)

In [10]:
df['ListingKey'].nunique()

90831

In [11]:
df['ListingNumber'].nunique()

90831

In [12]:
df['LoanKey'].nunique()

90831

In [13]:
df['LoanNumber'].nunique()

90831

In [14]:
df['MemberKey'].nunique()

90831

In [15]:
df['LoanStatus'] = df['LoanStatus'].replace(['Chargedoff',
 'Past Due (>120 days)'], ['Defaulted','Defaulted'])

In [16]:
df['LoanStatus'] = df['LoanStatus'].replace(['Completed' , 'Past Due (1-15 days)', 
       'Past Due (16-30 days)', 'Past Due (61-90 days)',
       'Past Due (31-60 days)', 'Past Due (91-120 days)','Cancelled'],
       ['Non-Defaulted','Non-Defaulted','Non-Defaulted',
       'Non-Defaulted','Non-Defaulted','Non-Defaulted','Non-Defaulted',])

In [17]:
non_null_ind = []
for i in df['ClosedDate']:
    if pd.isnull(i):
        continue
    else:
      non_null_ind.append(df[df['ClosedDate']==i].index )

In [18]:
for i in non_null_ind:
  df.loc[i,'LoanStatus']='Non-Defaulted'

In [19]:
non_deliquent_ind = []
for i in df['LoanCurrentDaysDelinquent']:
    if i>=180:
      non_deliquent_ind.append(df[df['LoanCurrentDaysDelinquent']==i].index )

In [20]:
for i in non_deliquent_ind:
  df.loc[i,'LoanStatus']='Defaulted'

In [21]:
df.drop(df[df['LoanStatus'] =='Current'].index,inplace=True)

In [22]:
df['LoanStatus'] = df['LoanStatus'].replace([ 'FinalPaymentInProgress'], ['Non-Defaulted'])

In [23]:
category_cols = df.select_dtypes(include=['category','object']).columns
# df[category_cols] = df[category_cols].fillna(df[category_cols].mode().iloc[0])

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
# df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

In [24]:
# importing sklearn
import sklearn
# importing simpleimputer
from sklearn.impute import SimpleImputer

In [25]:
#numeric columns
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(df[category_cols])
df[category_cols]= imputer.fit_transform(df[category_cols])
#categorical columns

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(df[numeric_cols])
df[numeric_cols]= imputer.fit_transform(df[numeric_cols])

## 1-Equated Monthly Installments (EMI)

* Tenure --> **Loan Tenure**
* Principle repayment --> **LP_CustomerPrinciplePayments**
* Interest --> BorrowerRate

In [26]:
emi_cols = ['LP_CustomerPayments' , 'LP_CustomerPrincipalPayments']
df[emi_cols].head()

,LP_CustomerPayments,LP_CustomerPrincipalPayments
0,11396.1400,9425.00
2,4186.6300,3001.00
11,1011.6500,1000.00
12,135.8174,88.64
15,521.1300,209.75


For each row in the dataset:
* Calculate result_1 = P* r * [(1-r)] ^n
* Calculate result_2 = [(1+r)]^n-1
* Calculate EMI = result1 / result2

In [27]:
def cal_EMI(P , r , n):
  P = P.values
  r = r.values
  n = n.values
  #print(P.shape[0])
  result_1 = np.empty(0)
  result_2 = np.empty(0)
  result = np.empty(0)
  for i in range(P.shape[0]):
    #print(P[i])
    #print(r[i])
    #print(n[i])
    #EMI = P x r x (1+r) ^ n / ((1+r)) ^ n - 1)
    #print(P[i] * (1+r[i]))
    result_1 = np.append(result_1 , P[i] * r[i] * np.power((1 + r[i]),n[i]))
    result_2 = np.append(result_2 , np.power((1 + r[i]) , n[i]) - 1)
    result =  np.append(result , (result_1[i] / result_2[i]))
 
  return result

In [28]:
df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129.0,2007-08-26 19:09:29.263000000,C,36.0,Non-Defaulted,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.00,0.00,0.0,1.0,0.0,0.0,0.0,258.0
2,0EE9337825851032864889A,81716.0,2007-01-05 15:00:47.090000000,HR,36.0,Non-Defaulted,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.00,0.00,0.0,1.0,0.0,0.0,0.0,41.0
11,0F1734025150298088A5F2B,213551.0,2007-10-09 20:28:33.640000000,C,36.0,Non-Defaulted,2008-01-07 00:00:00,0.15033,0.1325,0.1225,...,-0.88,0.0,0.00,0.00,0.0,1.0,0.0,0.0,0.0,53.0
12,0F1A3597143888805163EF7,1081604.0,2013-12-15 20:01:10.757000000,C,36.0,Non-Defaulted,2014-03-04 00:00:00,0.17969,0.1435,0.1335,...,-3.40,0.0,0.00,0.00,0.0,1.0,0.0,0.0,0.0,1.0
15,0F483544120452347F48121,577164.0,2012-04-10 09:14:46.297000000,C,36.0,Non-Defaulted,2012-12-19 00:00:00,0.35797,0.3177,0.3077,...,-9.81,0.0,3790.25,3790.25,0.0,1.0,0.0,0.0,0.0,10.0


In [29]:
df['ClosedDate']=pd.to_datetime(df['ClosedDate'])

In [30]:
df['𝐿𝑜𝑎𝑛𝐷𝑎𝑡𝑒𝑦𝑒𝑎𝑟'] = df['ClosedDate'].dt.year
df['month'] = df['ClosedDate'].dt.month

In [31]:
df['LoanOriginationDate']

0         2007-09-12 00:00:00
2         2007-01-17 00:00:00
11        2007-10-18 00:00:00
12        2014-01-07 00:00:00
15        2012-04-19 00:00:00
                 ...         
113914    2006-08-18 00:00:00
113918    2008-06-27 00:00:00
113923    2008-09-23 00:00:00
113926    2006-08-08 00:00:00
113929    2013-07-10 00:00:00
Name: LoanOriginationDate, Length: 42789, dtype: object

In [32]:
df['LoanOriginationDate']=pd.to_datetime(df['LoanOriginationDate'])

In [33]:
df['𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑦𝑒𝑎𝑟'] = df['LoanOriginationDate'].dt.year
df['𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑒𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑚𝑜𝑛𝑡ℎ'] = df['LoanOriginationDate'].dt.month

In [34]:
df['𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑒𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑚𝑜𝑛𝑡ℎ']

0          9
2          1
11        10
12         1
15         4
          ..
113914     8
113918     6
113923     9
113926     8
113929     7
Name: 𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑒𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑚𝑜𝑛𝑡ℎ, Length: 42789, dtype: int64

<h5>LoanTenure = (𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑦𝑒𝑎𝑟 − 𝐿𝑜𝑎𝑛𝐷𝑎𝑡𝑒𝑦𝑒𝑎𝑟) 𝑥 12 − (𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑒𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑚𝑜𝑛𝑡ℎ − 𝐿𝑜𝑎𝑛𝐷𝑎𝑡𝑒𝑚𝑜𝑛𝑡ℎ)</h5>

In [35]:
df['LoanTenure']=((df['𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑦𝑒𝑎𝑟']-df['𝐿𝑜𝑎𝑛𝐷𝑎𝑡𝑒𝑦𝑒𝑎𝑟'])*12)-(df['𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑒𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑚𝑜𝑛𝑡ℎ']-df['month'])

In [36]:
df['LoanTenure']

0        -25
2        -13
11       -21
12         2
15         8
          ..
113914   -17
113918   -41
113923   -36
113926   -11
113929   -17
Name: LoanTenure, Length: 42789, dtype: int64

In [37]:
df['EMI']=cal_EMI(df['LP_CustomerPrincipalPayments'],df['BorrowerRate'],df['LoanTenure'])

<ipython-input-27-4fc7b78a00c8>:17: RuntimeWarning: invalid value encountered in double_scalars
  result =  np.append(result , (result_1[i] / result_2[i]))
<ipython-input-27-4fc7b78a00c8>:17: RuntimeWarning: divide by zero encountered in double_scalars
  result =  np.append(result , (result_1[i] / result_2[i]))


In [38]:
df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors,𝐿𝑜𝑎𝑛𝐷𝑎𝑡𝑒𝑦𝑒𝑎𝑟,month,𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑦𝑒𝑎𝑟,𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑒𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑚𝑜𝑛𝑡ℎ,LoanTenure,EMI
0,1021339766868145413AB3B,193129.0,2007-08-26 19:09:29.263000000,C,36.0,Non-Defaulted,2009-08-14,0.16516,0.1580,0.1380,...,0.0,0.0,0.0,258.0,2009,8,2007,9,-25,-39.035716
2,0EE9337825851032864889A,81716.0,2007-01-05 15:00:47.090000000,HR,36.0,Non-Defaulted,2009-12-17,0.28269,0.2750,0.2400,...,0.0,0.0,0.0,41.0,2009,12,2007,1,-13,-36.629126
11,0F1734025150298088A5F2B,213551.0,2007-10-09 20:28:33.640000000,C,36.0,Non-Defaulted,2008-01-07,0.15033,0.1325,0.1225,...,0.0,0.0,0.0,53.0,2008,1,2007,10,-21,-10.482906
12,0F1A3597143888805163EF7,1081604.0,2013-12-15 20:01:10.757000000,C,36.0,Non-Defaulted,2014-03-04,0.17969,0.1435,0.1335,...,0.0,0.0,0.0,1.0,2014,3,2014,1,2,54.072767
15,0F483544120452347F48121,577164.0,2012-04-10 09:14:46.297000000,C,36.0,Non-Defaulted,2012-12-19,0.35797,0.3177,0.3077,...,0.0,0.0,0.0,10.0,2012,12,2012,4,8,74.875277


## 2- Preferred Return on Investment (PROI)



In [39]:
#the function displays a graph

def plot_by_woe(df_WoE , rotation_of_x_axis_labels = 0):
  x = np.array(df_WoE.iloc[:,0].apply(str))
  #turns the values of the column with index 0 to strings, makes an array from these strings, and passes it to variable x.
  y = df_WoE['WoE']
  #selects a column with label 'WoE' and passes it to variable y.
  plt.figure(figsize=(18,6))
  #sets the graph size to width 18 x height 6.
  plt.plot(x, y, marker = 'o',linestyle = '--',color = 'k')
  #Plots the datapoints with coordinates variable x on the x axisand variable y on the y axis.
  #sets the marker for each datapoint to a circle, the style line between the points to dashed,and the color to black.
  plt.xlabel(df_WoE.columns[0])
  #names the x axis with the name of the column with index 0.
  plt.ylabel('Weight of Evidence')
  #names the y axis 'weight of evidence'
  plt.title(str('Weight of Evidence by '+ df_WoE.columns[0]))
  #names the graph 'Weight of Evidence by ' the name of the column with index 0.
  plt.xticks(rotation = rotation_of_x_axis_labels)
  #rotates the labels of the x axis a predefined number of degrees.
  plt.show()

In [40]:
def PROI(df):
  #calculate ROI
  df['InterestAmount'] = (df['LoanOriginalAmount']*(df['BorrowerRate']))
  df['TotalAmount'] = (df['InterestAmount']+df['LoanOriginalAmount'])
  df['ROI'] = (df['InterestAmount']/df['TotalAmount'])

  #setting root
  df['PROI'] = df['ROI'].median()

  for i in range(df.shape[0]):
    #check out the creditGrade
    if df['CreditGrade'].iloc[i] == 'E':
      df['PROI'].iloc[i] = df['PROI'].iloc[i] + 0.05
    elif df['CreditGrade'].iloc[i] == 'HR':
      df['PROI'].iloc[i] = df['PROI'].iloc[i] + 0.05
    elif df['CreditGrade'].iloc[i] == 'A':
      df['PROI'].iloc[i] = df['PROI'].iloc[i] - 0.05
    elif df['CreditGrade'].iloc[i] == 'AA':
      df['PROI'].iloc[i] = df['PROI'].iloc[i] - 0.1

    #check out the TradesNeverDeliquent
    if df['TradesNeverDelinquent (percentage)'].iloc[i].all() < 0.7:
      df['PROI'].iloc[i] = df['PROI'].iloc[i] + 0.05


    #check out the AvailableBankCreditCardCredit
    if df['AvailableBankcardCredit'].iloc[i] <= 800:
       df['PROI'].iloc[i] = df['PROI'].iloc[i] + 0.05
    elif df['AvailableBankcardCredit'].iloc[i] >= 16500:
       df['PROI'].iloc[i] = df['PROI'].iloc[i] + 0.05

    #check out TotalInquiries
    if df['TotalInquiries'].iloc[i] > 28:
       df['PROI'].iloc[i] = df['PROI'].iloc[i] + 0.1
    elif df['TotalInquiries'].iloc[i] > 12:
       df['PROI'].iloc[i] = df['PROI'].iloc[i] + 0.05

      
  print(df['ROI'].describe())

  return df['PROI']


In [41]:
df['PROI']=PROI(df)
df.head()

<ipython-input-40-9470ce97be2f>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PROI'].iloc[i] = df['PROI'].iloc[i] + 0.05
<ipython-input-40-9470ce97be2f>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PROI'].iloc[i] = df['PROI'].iloc[i] + 0.05
<ipython-input-40-9470ce97be2f>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PROI'].iloc[i] = df['PROI'].iloc[i] - 0.1
<ipython-input-40-9470ce97be2f>:36: SettingWithCopyWa

count    42789.000000
mean         0.164418
std          0.055886
min          0.000000
25%          0.121651
50%          0.165972
75%          0.212536
max          0.332220
Name: ROI, dtype: float64


,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,𝐿𝑜𝑎𝑛𝐷𝑎𝑡𝑒𝑦𝑒𝑎𝑟,month,𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑦𝑒𝑎𝑟,𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑒𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑚𝑜𝑛𝑡ℎ,LoanTenure,EMI,InterestAmount,TotalAmount,ROI,PROI
0,1021339766868145413AB3B,193129.0,2007-08-26 19:09:29.263000000,C,36.0,Non-Defaulted,2009-08-14,0.16516,0.1580,0.1380,...,2009,8,2007,9,-25,-39.035716,1489.150,10914.150,0.136442,0.165972
2,0EE9337825851032864889A,81716.0,2007-01-05 15:00:47.090000000,HR,36.0,Non-Defaulted,2009-12-17,0.28269,0.2750,0.2400,...,2009,12,2007,1,-13,-36.629126,825.275,3826.275,0.215686,0.215972
11,0F1734025150298088A5F2B,213551.0,2007-10-09 20:28:33.640000000,C,36.0,Non-Defaulted,2008-01-07,0.15033,0.1325,0.1225,...,2008,1,2007,10,-21,-10.482906,132.500,1132.500,0.116998,0.165972
12,0F1A3597143888805163EF7,1081604.0,2013-12-15 20:01:10.757000000,C,36.0,Non-Defaulted,2014-03-04,0.17969,0.1435,0.1335,...,2014,3,2014,1,2,54.072767,574.000,4574.000,0.125492,0.165972
15,0F483544120452347F48121,577164.0,2012-04-10 09:14:46.297000000,C,36.0,Non-Defaulted,2012-12-19,0.35797,0.3177,0.3077,...,2012,12,2012,4,8,74.875277,1270.800,5270.800,0.241102,0.215972


## 3-Elgible Loan Amount (ELA)

Components of ELA:
 
 * A: 'AppliedAmount' --> **LoanOriginalAmount**
 * R: 'Interest' --> **BorrowerRate**
 * N: 'LoanTenure' --> **LoanTenure**
 * I: 'IncomeTotal' --> **StatedMonthlyIncome**

In [42]:
ela_cols = ['DebtToIncomeRatio' , 'IncomeRange' , 'IncomeVerifiable' , 'StatedMonthlyIncome']

df[ela_cols].head()

,DebtToIncomeRatio,IncomeRange,IncomeVerifiable,StatedMonthlyIncome
0,0.17,"$25,000-49,999",True,3083.333333
2,0.06,Not displayed,True,2083.333333
11,0.27,"$1-24,999",True,1666.666667
12,0.18,"$25,000-49,999",True,2416.666667
15,0.49,"$50,000-74,999",True,5500.000000


In [43]:
df_new = df[df['IncomeVerifiable'] == True]

print(df_new.shape)
print(df.shape[0] - df_new.shape[0])

(39450, 91)
3339


Calculation Procedure for each row in the dataset: 

* Calculate: Total Payment Due = (A + (Ar))n
* Calculate: Max allowable amount = I * 12 * 30%
* If(total Payment Due <= Max allowable amount) Then ELA = AppliedAmount Else ELA = Max allowable amount

In [44]:
def calculate_total_payment_due(A, Ar, n):
    return (A + (Ar))* n

def calculate_max_allowable_amount(I):
    return I * 12 * 0.3

def calculate_ela(total_payment_due, max_allowable_amount, applied_amount):
    if total_payment_due <= max_allowable_amount:
        return applied_amount
    else:
        return max_allowable_amount


ela_values = []  # Initialize an empty list to store ela values

for i in range(len(df_new)):
    row = df_new.iloc[i]
    total_payment_due = calculate_total_payment_due(row['LoanOriginalAmount'], row['LoanOriginalAmount']*row['BorrowerRate'], row['LoanTenure'])
    max_allowable_amount = calculate_max_allowable_amount(row['StatedMonthlyIncome'])
    ela = calculate_ela(total_payment_due, max_allowable_amount, row['LoanOriginalAmount'])
    ela_values.append(ela)  # Append ela value to the list

df_new['EligibleLoanAmount'] = ela_values  # Assign the list of ela values to the column

print(df_new['EligibleLoanAmount'])

   

0          9425.000000
2          3001.000000
11         1000.000000
12         8700.000001
15        19800.000000
              ...     
113914     3000.000000
113918     7000.000000
113923     8000.000000
113926     3000.000000
113929     2500.000000
Name: EligibleLoanAmount, Length: 39450, dtype: float64


<ipython-input-44-e0ae3206f0d0>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['EligibleLoanAmount'] = ela_values  # Assign the list of ela values to the column


In [45]:
df_new.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,month,𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑦𝑒𝑎𝑟,𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑒𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑚𝑜𝑛𝑡ℎ,LoanTenure,EMI,InterestAmount,TotalAmount,ROI,PROI,EligibleLoanAmount
0,1021339766868145413AB3B,193129.0,2007-08-26 19:09:29.263000000,C,36.0,Non-Defaulted,2009-08-14,0.16516,0.1580,0.1380,...,8,2007,9,-25,-39.035716,1489.150,10914.150,0.136442,0.165972,9425.000000
2,0EE9337825851032864889A,81716.0,2007-01-05 15:00:47.090000000,HR,36.0,Non-Defaulted,2009-12-17,0.28269,0.2750,0.2400,...,12,2007,1,-13,-36.629126,825.275,3826.275,0.215686,0.215972,3001.000000
11,0F1734025150298088A5F2B,213551.0,2007-10-09 20:28:33.640000000,C,36.0,Non-Defaulted,2008-01-07,0.15033,0.1325,0.1225,...,1,2007,10,-21,-10.482906,132.500,1132.500,0.116998,0.165972,1000.000000
12,0F1A3597143888805163EF7,1081604.0,2013-12-15 20:01:10.757000000,C,36.0,Non-Defaulted,2014-03-04,0.17969,0.1435,0.1335,...,3,2014,1,2,54.072767,574.000,4574.000,0.125492,0.165972,8700.000001
15,0F483544120452347F48121,577164.0,2012-04-10 09:14:46.297000000,C,36.0,Non-Defaulted,2012-12-19,0.35797,0.3177,0.3077,...,12,2012,4,8,74.875277,1270.800,5270.800,0.241102,0.215972,19800.000000


In [99]:
cols_emi= ['LP_CustomerPayments' , 'LP_CustomerPrincipalPayments',
        '𝐿𝑜𝑎𝑛𝐷𝑎𝑡𝑒𝑦𝑒𝑎𝑟','month','𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑦𝑒𝑎𝑟','𝑀𝑎𝑡𝑢𝑟𝑖𝑡𝑦𝐷𝑎𝑡𝑒𝑂𝑟𝑖𝑔𝑖𝑛𝑎𝑙𝑚𝑜𝑛𝑡ℎ',
        'LP_CustomerPrincipalPayments','BorrowerRate']

In [100]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [101]:
df_model = df_new

In [102]:
df_model=df_model.fillna(0)

In [103]:
df_model['EMI'].replace([np.inf, -np.inf], np.nan, inplace=True)

In [104]:
df_model['EMI'].isna().any()

True

In [105]:
df_model=df_model.fillna(0)

In [106]:
#Assign X and y
X = df_model[cols_emi].values
y = df_model['EMI'].values

In [107]:
#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Evaluating the model 
print("Training accuracy for EMI=", lr.score(X_train, y_train))
print("Testing accuracy for EMI=", lr.score(X_test, y_test))

Training accuracy for EMI= 0.22065791717398986
Testing accuracy for EMI= 0.21881099714470198


In [114]:
cols_roi=['LoanOriginalAmount','BorrowerRate'
,'TradesNeverDelinquent (percentage)',
'AvailableBankcardCredit','TotalInquiries'
,'InterestAmount','TotalAmount','StatedMonthlyIncome', 'DebtToIncomeRatio']

In [117]:
#Assign X and y
X = df_model[cols_roi].values
y = df_model['ROI'].values

In [118]:
#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Evaluating the model 
print("Training accuracy for ROI=", lr.score(X_train, y_train))
print("Testing accuracy for ROI=", lr.score(X_test, y_test))

Training accuracy for ROI= 0.996435893446353
Testing accuracy for ROI= 0.996457038317881


In [122]:
cols_ela=['DebtToIncomeRatio'  , 
          'IncomeVerifiable' , 'StatedMonthlyIncome',
          'LoanOriginalAmount','StatedMonthlyIncome',
          'BorrowerRate','LoanTenure']

In [123]:
#Assign X and y
X = df_model[cols_ela].values
y = df_model['EligibleLoanAmount'].values

In [124]:
#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Fit linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Evaluating the model 
print("Training accuracy for ELA=", lr.score(X_train, y_train))
print("Testing accuracy for ELA=", lr.score(X_test, y_test))

Training accuracy for ELA= 0.7540640683618861
Testing accuracy for ELA= 0.742537658418467
